In [ ]:
%%manim -v WARNING -qh OverfitUnderfit
from manim import *
import numpy as np

class OverfitUnderfit(Scene):
    def construct(self):
        # 数据
        x_values = np.linspace(1, 8.5, 9)
        y_values = 0.8 * x_values**2 - 6 * x_values + 19
        noisy_y = y_values + np.random.uniform(-3, 3, size=len(x_values))

        # =============== 欠拟合场景 ===============
        # 坐标轴
        axes_left = Axes(
            x_range=[0, 10, 2],
            y_range=[0, 30, 5],
            x_length=4,
            y_length=4,
            axis_config={"color": BLUE, "include_numbers": True, "font_size": 18},
        ).to_edge(LEFT).shift(DOWN*0.5+RIGHT*1.0).scale(1.3)
        # 数据点
        dots_left = VGroup(*[
            Dot(axes_left.c2p(x, y), color=YELLOW, radius=0.05)
            for x, y in zip(x_values, noisy_y)
        ])
        # 1 次拟合
        coeffs_1 = np.polyfit(x_values, noisy_y, 1)
        func_1 = lambda x: coeffs_1[0]*x + coeffs_1[1]
        linear_fit = axes_left.plot(func_1, color=BLUE)
        # 文字（欠拟合）
        underfit_text = VGroup(
            Text("欠拟合（Underfitting）", font="Software Yahei", color=BLUE).scale(0.9),
            Text("起因：", font="Software Yahei", color=ORANGE).scale(0.7),
            Text("• 模型过于简单", font="Software Yahei").scale(0.5),
            Text("• 学习不足/特征不足", font="Software Yahei").scale(0.5),
            Text("解决方法：", font="Software Yahei", color=ORANGE).scale(0.7),
            Text("✓ 增加模型复杂度", font="Software Yahei").scale(0.5),
            Text("✓ 增加特征/训练轮次", font="Software Yahei").scale(0.5)
        ).arrange(DOWN, aligned_edge=LEFT).to_edge(RIGHT).shift(UP*0.5+LEFT*0.7)
        # 展示欠拟合
        self.play(Create(axes_left), FadeIn(dots_left))
        self.play(Create(linear_fit))
        self.play(LaggedStartMap(FadeIn, underfit_text, shift=RIGHT))
        self.wait(4)
        # 黑屏过渡
        self.play(FadeOut(Group(*self.mobjects)), run_time=2)
        self.wait(1)

        # =============== 过拟合场景 ===============
        axes_right = Axes(
            x_range=[0, 10, 2],
            y_range=[0, 30, 5],
            x_length=4,
            y_length=4,
            axis_config={"color": BLUE, "include_numbers": True, "font_size": 18},
        ).to_edge(LEFT).shift(DOWN*0.5+RIGHT*1.0).scale(1.3)

        dots_right = VGroup(*[
            Dot(axes_right.c2p(x, y), color=YELLOW, radius=0.05)
            for x, y in zip(x_values, noisy_y)
        ])

        # 12 次拟合
        coeffs_12 = np.polyfit(x_values, noisy_y, 9)
        func_12 = lambda x: np.polyval(coeffs_12, x)
        poly12_fit = axes_right.plot(func_12, color=RED)

        # 文字（过拟合）
        overfit_text = VGroup(
            Text("过拟合（Overfitting）", font="Software Yahei", color=RED).scale(0.9),
            Text("起因：", font="Software Yahei", color=ORANGE).scale(0.7),
            Text("• 模型过于复杂", font="Software Yahei").scale(0.5),
            Text("• 数据量不足/噪声大", font="Software Yahei").scale(0.5),
            Text("解决方法：", font="Software Yahei", color=ORANGE).scale(0.7),
            Text("✓ 正则化（L1/L2, Dropout）", font="Software Yahei").scale(0.5),
            Text("✓ 增加数据/早停法", font="Software Yahei").scale(0.5)
        ).arrange(DOWN, aligned_edge=LEFT).to_edge(RIGHT).shift(UP*0.5+LEFT*0.7)

        # 展示过拟合
        self.play(Create(axes_right), FadeIn(dots_right))
        self.play(Create(poly12_fit))
        self.play(LaggedStartMap(FadeIn, overfit_text, shift=RIGHT))
        self.wait(6)

        # =============== 突出"正则化" ===============
        # 拿到"正则化"这行（在 overfit_text[5] 里）
        regularization_line = overfit_text[5]
        # 先改成蓝色
        self.play(regularization_line.animate.set_color(BLUE))
        self.wait(1)
        # 提取"正则化"三个字
        keyword = Text("正则化", font="Microsoft YaHei", color=BLUE).scale(1.2)
        keyword.move_to(regularization_line.get_center())
        # 替换成"正则化"这三个字
        self.play(Transform(regularization_line, keyword))
        # 其他元素
        others = Group(*[m for m in self.mobjects if m is not regularization_line])
        # 让"正则化"一边放大，一边移动到上方
        self.play(
            regularization_line.animate.scale(1.5).move_to(UP).shift(UP*1.0),
            FadeOut(others)   # 同时其他东西淡出
        )
        self.wait(2)
        # 正则化下面的具体方法
        methods = VGroup(
            Text("• L1 正则化（Lasso）", font="Microsoft YaHei", color=WHITE).scale(0.7),
            Text("• L2 正则化（Ridge）", font="Microsoft YaHei", color=WHITE).scale(0.7),
            Text("• 随机失活（Dropout）", font="Microsoft YaHei", color=WHITE).scale(0.7),
            Text("• 早停法（Early Stopping）", font="Microsoft YaHei", color=WHITE).scale(0.7),
            Text("• 数据增强（Data Augmentation）", font="Microsoft YaHei", color=WHITE).scale(0.7),
        ).arrange(DOWN, aligned_edge=LEFT, buff=0.3)  # 0.3 调整行间距

        # 放到"正则化"下面
        methods.next_to(regularization_line, DOWN, buff=0.5).shift(RIGHT*1.5)

        # 渐显出现
        self.play(LaggedStartMap(FadeIn, methods, shift=DOWN))
        self.wait(3)